In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# YOLOv7 설치

In [2]:
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0) 
Pytorch version: 2.1.0+cu118 


In [3]:
!nvidia-smi

Tue Oct 24 23:33:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Total 1191 (delta 0), reused 0 (delta 0), pack-reused 1191
Receiving objects: 100% (1191/1191), 74.22 MiB | 28.04 MiB/s, done.
Resolving deltas: 100% (515/515), done.
/content/yolov7
cfg	detect.py  hubconf.py  models	  requirements.txt  tools	  utils
data	export.py  inference   paper	  scripts	    train_aux.py
deploy	figure	   LICENSE.md  README.md  test.py	    train.py


In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00


## MyDrive 에 data 라는 이름으로 train 이랑 val 각각 images labels 파일 zip 형태로 업로드 해놓기

In [6]:
import zipfile

# 압축 해제할 파일 경로
zip_file_path = '/content/drive/MyDrive/data.zip'

# 압축 해제될 폴더 경로 (원하는 경로로 변경 가능)
extracted_folder_path = '/content/yolov7/data'

# 압축 해제 실행
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print("압축 해제가 완료되었습니다.")

압축 해제가 완료되었습니다.


## 학습 전에 yaml 파일 넣기 (2가지)

*   yolov7 -> cfg -> training 폴더 안에 yolov7-custom.yaml 넣기
*   yolov7 -> data 폴더 안에 custom_data.yaml 넣기



In [7]:
!python train.py --workers  1 --device 0 --batch-size 16 --epochs 50 --img 640 640 --data data/custom_data.yaml --hyp data/hyp.scratch.custom.yaml --cfg cfg/training/yolov7-custom.yaml --name yolov7-custom --weights yolov7.pt


2023-10-24 23:39:26.251748: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-24 23:39:26.251803: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-24 23:39:26.251840: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-24 23:39:26.259662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 23:39:27.229025: W tensorflow/c

In [ ]:
!# Detection
!python detect.py --weights /content/yolov7/runs/train/yolov7-custom3/weights/best.pt --conf 0.25 --img-size 640 --source  /content/tc.png

Namespace(weights=['/content/yolov7/runs/train/yolov7-custom3/weights/best.pt'], source='/content/tc.png', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.1.0+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36535712 parameters, 6194944 gradients, 103.3 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 

In [ ]:
# 모든 파일 삭제
import shutil
shutil.rmtree('/content/drive/MyDrive/yolov7-main')
#shutil.rmtree('/content/C:/Users/Administrator/Desktop/(icms)디지털스마트부산아카데미/객체탐지 연습/도로장애물·표면 인지 영상(수도권 외)/Training/labels')